In [1]:
import osmnx as ox
import matplotlib as plt
import geopandas as gpd
from k_means_constrained import KMeansConstrained
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model



In [2]:
# load in Data
Porto_data = pd.read_csv(r'C:\Users\NimaMehrafar\UC\UC_Project\porto_data_2')
SanFrancisco_data = pd.read_csv(r'C:\Users\NimaMehrafar\UC\UC_Project\san_francisco_data_2')
Tokyo_data = pd.read_csv(r'C:\Users\NimaMehrafar\UC\UC_Project\tokyo_data_2')

In [3]:
Tokyo_data

,Unnamed: 0.1,Unnamed: 0,highway,geometry,crossing,name,n_type,osmid
0,0,0,crossing,POINT (139.4504331 35.614965),traffic_signals,NaN,4,0
1,1,1,crossing,POINT (139.4081901 35.6135793),zebra,NaN,4,1
2,2,2,bus_stop,POINT (139.4159142 35.7440937),NaN,中北台,1,2
3,3,3,bus_stop,POINT (139.4161435 35.7422084),NaN,上北台団地東,1,3
4,4,4,bus_stop,POINT (139.417002 35.7374482),NaN,桜街道,6,4
...,...,...,...,...,...,...,...,...
60817,60817,60817,crossing,POINT (139.8052154 35.7356922),marked,NaN,5,60817
60818,60818,60818,crossing,POINT (139.8050389 35.7355167),marked,NaN,2,60818
60819,60819,60819,crossing,POINT (139.805133 35.7355321),marked,NaN,5,60819
60820,60820,60820,bus_stop,POINT (139.8593339 35.6473447),NaN,臨海車庫前,6,60820


In [4]:
clf = KMeansConstrained(
     n_clusters=85,
    size_min=40,
    size_max=60, 
    random_state=0)

In [5]:
#Preprocessing
Porto_data = Porto_data[['element_type','highway','n_type']]
label_encoder = LabelEncoder()
Porto_data['highway'] = label_encoder.fit_transform(Porto_data['highway'])
Porto_data['element_type'] = label_encoder.fit_transform(Porto_data['element_type'])


In [6]:
#Clustering
group_size = 30
clf.fit_predict(Porto_data.values)
clusters = clf.labels_
Porto_data['cluster'] = clusters

In [7]:
# X and Y
X = Porto_data[['element_type','highway']]
y = Porto_data['n_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
unique_labels = set(y_train)
print("Unique Labels:", unique_labels)
y_train = y_train - 1
y_test = y_test - 1


Unique Labels: {1, 2, 3, 4, 5}


In [9]:
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))  # Use 'sigmoid' for binary classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)




Epoch 1/10


83/83 [==============================] - 5s 16ms/step - loss: 1.4396 - accuracy: 0.4307 - val_loss: 1.2615 - val_accuracy: 0.7221
Epoch 2/10
83/83 [==============================] - 1s 8ms/step - loss: 1.0674 - accuracy: 0.6641 - val_loss: 1.0161 - val_accuracy: 0.7221
Epoch 3/10
83/83 [==============================] - 1s 9ms/step - loss: 0.8855 - accuracy: 0.7136 - val_loss: 0.8565 - val_accuracy: 0.7221
Epoch 4/10
83/83 [==============================] - 1s 11ms/step - loss: 0.8273 - accuracy: 0.7136 - val_loss: 0.8407 - val_accuracy: 0.7221
Epoch 5/10
83/83 [==============================] - 1s 9ms/step - loss: 0.8029 - accuracy: 0.7140 - val_loss: 0.8371 - val_accuracy: 0.6964
Epoch 6/10
83/83 [==============================] - 1s 8ms/step - loss: 0.8011 - accuracy: 0.7140 - val_loss: 1.1324 - val_accuracy: 0.6677
Epoch 7/10
83/83 [==============================] - 1s 10ms/step - loss: 0.7919 - accuracy: 0.7136 - val_loss: 0.9308 - val_accuracy: 0.6586
Epoch 8/10
83

In [10]:
y_pred = model.predict(X_test)
_, accuracy = model.evaluate(X_test, y_test)
accuracy

26/26 [==============================] - 0s 5ms/step - loss: 0.8325 - accuracy: 0.7415


0.7415459156036377

In [11]:
embedding_model = Model(inputs=model.input, outputs=model.layers[-2].output)


In [12]:
embeddings = embedding_model.predict(X_train)

104/104 [==============================] - 1s 4ms/step


In [14]:
embeddings[0]

array([-1.550345  , -0.28344545, -0.7479494 , -0.24462476, -2.2941072 ,
       -0.3157131 , -0.58305645,  4.338916  ,  4.734445  ,  4.3688865 ,
        0.3321503 , -0.3260085 ,  3.663627  , -0.49686533, -0.24679339,
       -0.30266222, -1.722468  , -1.5876964 ,  3.9059036 ,  4.2728972 ,
        4.871971  , -0.6356286 , -1.2031636 ,  4.182301  ,  4.055821  ,
        3.859782  ,  4.569083  ,  4.021937  , -1.5407001 ,  2.9428232 ,
        3.1956816 ,  2.5938122 ], dtype=float32)